In [5]:
import os
import sys
sys.path.append(os.getcwd()+'/..')

# from pdearena import utils
# import torch
# from pdearena.data.datamodule import PDEDataModule
# from pdearena.lr_scheduler import LinearWarmupCosineAnnealingLR  # noqa: F401
# from pdearena.models.pdemodel import PDEModel

from helper_functions.helper_functions.data import write_h5, read_h5_numpy
from helper_functions.helper_functions.general import touch_dir
from time import time
import numpy as np
import h5py
from tqdm import tqdm
import shutil
import _pickle as cPickle
import pickle 
import hdf5plugin


In [6]:
import sys
sys.path


['/Users/matte/Documents/RESEARCH/FORKED/sci_ML/pdearena/pdedatagen',
 '/Users/matte/miniforge3/envs/pdearena-dev/lib/python39.zip',
 '/Users/matte/miniforge3/envs/pdearena-dev/lib/python3.9',
 '/Users/matte/miniforge3/envs/pdearena-dev/lib/python3.9/lib-dynload',
 '',
 '/Users/matte/miniforge3/envs/pdearena-dev/lib/python3.9/site-packages',
 '/Users/matte/Documents/RESEARCH/FORKED/sci_ML/pdearena/pdedatagen/..',
 '/Users/matte/Documents/RESEARCH/FORKED/sci_ML/pdearena/pdedatagen/..',
 '/Users/matte/Documents/RESEARCH/FORKED/sci_ML/pdearena/pdedatagen/..',
 '/Users/matte/Documents/RESEARCH/FORKED/sci_ML/pdearena/pdedatagen/..',
 '/Users/matte/Documents/RESEARCH/FORKED/sci_ML/pdearena/pdedatagen/..']

In [7]:
def hdf5_to_dict(file):
    d = dict()
    for key,val in file.items():
        if type(val) == h5py._hl.dataset.Dataset:
            d[key] = np.array(val)
        else:
            d[key] = hdf5_to_dict(val)
    return d

In [5]:
### Chosen the following saving strategies
'''
- hdf5plugin.Blosc(cname='lz4', clevel = 1)
- pickle
'''
compressor = hdf5plugin.Blosc(cname='lz4', clevel = 1)

input_folder = '../pdearena_data/NavierStokes-2D-64x64/'
target_folders = [
    '../pdearena_data/NavierStokes-2D-64x64_pkl/',
]
files = os.listdir(input_folder)


def convert_file(filein,fileout):
    with h5py.File(filein, 'r') as hf_source:
        d = hdf5_to_dict(hf_source)
        if 'pkl' in fileout:
            with open(fileout,"wb") as filehandler:
                cPickle.dump(d,filehandler,protocol=pickle.HIGHEST_PROTOCOL)
            # pass
        else:
            pass
            # with h5py.File(fileout, "w") as h5f_target:
            #     mode = next(iter(d.keys()))
            #     dataset = h5f_target.create_group(mode)
            #     for k,v in d[mode].items():
            #         # print('processing {}'.format(k))
            #         dataset.create_dataset(
            #             k, 
            #             data = v,
            #             shape = v.shape,
            #             **compressor
            #         )


for target_folder in target_folders:
    touch_dir(target_folder)
    
    for file in tqdm(files,total=len(files)):
        try:
            if 'yaml' not in file:
                fileout = file
                if 'pkl' in target_folder:
                    fileout = fileout[:-3] + '.pkl'
                convert_file(
                    os.path.join(input_folder,file),
                    os.path.join(target_folder,fileout)
                )
            else:
                fileout = file
                shutil.copyfile(
                    os.path.join(input_folder,file),
                    os.path.join(target_folder,fileout)
                )
        except:
            print(file,fileout)
        



100%|██████████| 310/310 [00:06<00:00, 45.11it/s]


In [8]:
def load_pickle(filename):
    with open(filename,'rb') as filehandler:
        return cPickle.load(filehandler)

def save_pickle(data, filename):
    with open(filename,"wb") as filehandler:
        cPickle.dump(data,filehandler,protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
### Save 1 training sample per pickle

compressor = hdf5plugin.Blosc(cname='lz4', clevel = 1)

# input_folder = '../pdearena_data/NavierStokes-2D-32x32_pkl/'
# target_folders = [
#     '../pdearena_data/NavierStokes-2D-32x32_pkl_single/',
# ]

input_folder = '../pdearena_data/NavierStokes-2D-64x64_pkl/'
target_folders = [
    '../pdearena_data/NavierStokes-2D-64x64_pkl_single/',
]
files = os.listdir(input_folder)

here = os.getcwd()


# def convert_file(filein,fileout):
#     with h5py.File(filein, 'r') as hf_source:
#         d = hdf5_to_dict(hf_source)
#         if 'pkl' in fileout:
#             with open(fileout,"wb") as filehandler:
#                 cPickle.dump(d,filehandler,protocol=pickle.HIGHEST_PROTOCOL)
#             # pass
#         else:
#             pass
#             # with h5py.File(fileout, "w") as h5f_target:
#             #     mode = next(iter(d.keys()))
#             #     dataset = h5f_target.create_group(mode)
#             #     for k,v in d[mode].items():
#             #         # print('processing {}'.format(k))
#             #         dataset.create_dataset(
#             #             k, 
#             #             data = v,
#             #             shape = v.shape,
#             #             **compressor
#             #         )


for target_folder in target_folders:
    touch_dir(target_folder)
    
    for file in tqdm(files,total=len(files)):
        try:
            if 'yaml' not in file:

                d = load_pickle(
                    os.path.join(input_folder,file)
                )

                mode = next(iter(d.keys()))

                for i in range(d[mode]['u'].shape[0]):
                    d_i = {}
                    d_i[mode] = {}

                    for k in d[mode].keys():
                        d_i[mode][k] = d[mode][k][i,...]
                        d_i[mode][k] = d_i[mode][k][np.newaxis,...] 
                        # print(k,d[mode][k].shape)
                        # print(k,d_i[mode][k].shape)

                    file_i = file.replace('.pkl','') + '_' + str(i) + '.pkl'
                    file_i = os.path.join(target_folder,file_i)
                    file_i = os.path.join(
                        os.getcwd(),
                        file_i
                    )
                    with open(file_i,"wb") as filehandler:
                        cPickle.dump(d_i,filehandler,protocol=pickle.HIGHEST_PROTOCOL)

        except:
            print(file,fileout)
        



100%|██████████| 312/312 [00:09<00:00, 34.10it/s]
